In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Slide Generation with Gemini and Marp

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fproductivity%2Fslide_generation_with_marp.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/productivity/slide_generation_with_marp.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| Author(s) | [Paige Bailey](https://github.com/dynamicwebpaige), [Holt Skinner](https://github.com/holtskinner) |

## Overview

[Marp](https://marp.app/) is a framework for creating slide decks using Markdown.

This notebook shows how to use the Gemini API and the Google Gen AI SDK for Python to convert a blog post into a slide deck overview.

You will complete the following tasks:

- Use [Gitingest](https://gitingest.com/) to load the Marp repository as a large string.
- Send the Marp repository and a blog post URL to the Gemini API.
- Parse the markdown output from Gemini
- Send markdown to [Marp CLI](https://github.com/marp-team/marp-cli) to convert to PDF.

## Getting Started

### Install Google Gen AI SDK for Python


In [ ]:
%pip install --upgrade --quiet google-genai gitingest

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Connect to a generative AI API service

Google Gen AI APIs and models including Gemini are available in the following two API services:

- **[Google AI for Developers](https://ai.google.dev/gemini-api/docs)**: Experiment, prototype, and deploy small projects.
- **[Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs)**: Build enterprise-ready projects on Google Cloud.

The Google Gen AI SDK provides a unified interface to these two API services.

This notebook shows how to use the Google Gen AI SDK with the Gemini API in Vertex AI.

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

In [ ]:
from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries


In [ ]:
import re

from IPython.display import Markdown, display
from gitingest import ingest
from google.genai.types import GenerateContentConfig, Part
import nest_asyncio

nest_asyncio.apply()

### Load the Gemini 2.0 Flash Thinking model

Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [ ]:
MODEL_ID = "gemini-2.0-flash-thinking-exp-01-21"  # @param {type: "string"}

### Load the Marp repository

In [ ]:
repo_dir = "https://github.com/marp-team/marp"

summary, tree, content = ingest(source=repo_dir, branch="main")

### Send the Marp codebase and blog post in prompt to Gemini

In [ ]:
PROMPT = """
Please create a slide overview version of this website (about DeepMind's AI Co-scientist), using the same kind of style and formatting. Link to all of the images in the Marp deck (especially .mp4 files and .GIF). I've also included the Marp codebase, so you can refer back to it for design changes. Make sure that the text is appropriately sized for each slide (keep it quite small, or divide up extra long text into multiple slides), and make sure that the images are placed in well-designed locations.
"""

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        content,
        PROMPT,
        Part.from_uri(
            file_uri="https://research.google/blog/accelerating-scientific-breakthroughs-with-an-ai-co-scientist/",
            mime_type="text/html",
        ),
    ],
    config=GenerateContentConfig(
        system_instruction="Only output markdown.",
        max_output_tokens=8192,
    ),
)

display(Markdown(response.text))

### Parse response from Gemini

In [ ]:
marp_text = re.findall(r"```(?:\w*\n)?(.*?)```", response.text + "\n```", re.DOTALL)[0]

with open("slide-deck.md", "w") as f:
    f.write(marp_text)

### Convert Markdown to a PDF

> **NOTE:** This will only work on a machine with a web browser installed. Colab is not supported.

In [ ]:
!npx @marp-team/marp-cli@latest slide-deck.md --pdf